In [1]:
import datetime
import pandas as pd
import quandl
import matplotlib.pyplot as plt
import math
from quandl import QuandlError
plt.style.use('alex')

In [2]:
WIKIP2000 = pd.read_csv('C:/Users/awright/Desktop/January 2018/Other/trading/Market Histories/WIKIP2000_20170101.csv', encoding = "ISO-8859-1")

In [3]:
WIKIP2000.head()

,Unnamed: 0,Symbol,Description,Date,Volume,Open,Low,High,Close,DailyPctChg
0,0,BRK_B,BRK_B,2017-01-03,4090967.0,164.34,162.44,164.71,163.83,NaN
1,1,BRK_B,BRK_B,2017-01-04,3568539.0,164.45,163.00,164.57,164.08,0.001526
2,2,BRK_B,BRK_B,2017-01-05,2982446.0,164.06,162.18,164.14,163.30,-0.004754
3,3,BRK_B,BRK_B,2017-01-06,2697027.0,163.44,162.64,163.80,163.41,0.000674
4,4,BRK_B,BRK_B,2017-01-09,3563591.0,163.04,162.02,163.25,162.02,-0.008506


In [ ]:
class Market:
    ''' The Market class has two primary roles: to pull data from the Quandl API, and to store that data in a
    structured dataframe, self.History.  The one required parameter is exchange -- which will correspond to one
    of the available exchange codes listed below.  User can also pass a datestart parameter, which specifies from
    which date onwards the request for data will be made. The passed exchange code will be formatted correctly for the
    Quandl API and passed for a request.  
    
    The file parameter should be the location of the file that stores your market data in .csv format, and the data 
    should be structured in the format of the class self.History pandas DataFrame. In the event that a file
    parameter is passed, the initialization of the class will identify the latest date in the dataframe, and set
    datestart to this date.
    
    In all cases the class pulls in data up to the current date.
    
    Market class has access to the following exchanges via the Quandl API:
    
    Wiki stock Prices (High Volumne)	WIKIP2000
     Wiki stock Prices (Lower Volumne)	WIKIP_over2000
    BraveNewCoin	BNC
    London Metal Exchange	LME
    Chicago Mercantile Exchange	CME
    Intercontinental Exchange Futures	ICE
    London Stock Exchange	LSE
    Wiki Commodity Price	COM
    Hong Kong Exchange	HKEX
    Warsaw Stock Exchange	WSE
    Global Petroleum Prices (Gasoline)	GPPG
    Global Petroleum Prices (Diesel)	GPPD
    London Platinum and Palladium Market	LPPM
    LIFFE Futures Data	LIFFE
    Eurex Futures	EUREX
    Johnson Matthey	JOHNMATT
'''
    
    def __init__(self,
                 file = '',
                 ):
        
        quandl.ApiConfig.api_key = 'Npr_WVEspvDGSfRd9Dt_'          
        self.Dir = pd.read_csv("C:/Users/awright/Desktop/January 2018/Other/trading/Asset listings/MarketandListingCodes.csv")
        self.colors = [prop['color'] for prop in plt.rcParams['axes.prop_cycle']]
        

        
        if file != '':
################     Append an existing History from the latest date to today ################

            self.History = pd.read_csv(file, index_col = 0, encoding = "ISO-8859-1", dtype = {'Symbol':object,
                                                                                    'Description':object,
                                                                                    'Date': str,
                                                                                    'Open':'float64',
                                                                                    'Close':'float64',
                                                                                    'High':'float64',
                                                                                    'Low':'float64',
                                                                                    'Volume':'float64',
                                                                                    'DailyPctChg':'float64'},parse_dates=['Date'])  
            self.dateStart = max(self.History.Date)
        
        else:

            self.History = pd.DataFrame([],columns = ['Symbol',
                                                      'Description',
                                                      'Date',
                                                      'Open',
                                                      'Close',
                                                      'High',
                                                      'Low',
                                                      'Volume',
                                                      'DailyPctChg']).astype({'Symbol':object,
                                                                         'Description':object,
                                                                         'Date': datetime.date,
                                                                         'Open':'float64',
                                                                         'Close':'float64',
                                                                         'High':'float64',
                                                                         'Low':'float64',
                                                                         'Volume':int,
                                                                         'DailyPctChg':'float64'})
            
    def getHistory(self,
                  exchange,
                  period = 'daily',
                  datestart = (str((datetime.datetime.now().date() + datetime.timedelta(days  =-1)).year) 
                      + '-' + str((datetime.datetime.now().date() + datetime.timedelta(days  =-1)).month) + '-' 
                      + str((datetime.datetime.now().date() + datetime.timedelta(days  =-1)).day)),
                  dateend = (str((datetime.datetime.now().date().year))
                      + '-' + str((datetime.datetime.now().date().month))
                      + '-' + str((datetime.datetime.now().date().day)))):
        
        self.dateStart = datestart
        self.dateEnd = dateend
        
        if len(self.History) != 0:
            self.dateStart = max(self.History.Date) + datetime.timedelta(days=1)

        if exchange == 'WIKIP2000':
            syms = pd.read_csv("C:/Users/awright/Desktop/January 2018/Other/trading/Asset listings/StockSymbols2000.csv"
                              , encoding = "ISO-8859-1")
            
            for i in range(len(syms)):  
                sym = syms['Symbol'][i]
                description = syms['Description'][i]
                try:
                    code = 'WIKI/' + sym
                    assetHistory = quandl.get(code, start_date=self.dateStart, end_date=self.dateEnd,collapse = period).reset_index()
                    assetHistory = assetHistory[['Date','Adj. Open','Adj. High','Adj. Low','Adj. Close','Adj. Volume']]
                    assetHistory['Symbol'] = sym
                    assetHistory['Description'] = description
                    assetHistory.columns = ['Date','Open','High','Low','Close', 'Volume','Symbol','Description']

                    self.History = pd.concat([self.History,assetHistory], ignore_index=True)
                    print('{} data acquired for dates from {} to {}'.format(description,self.dateStart,self.dateEnd))

                except (KeyError, NameError):
                    print('{} data FAILED TO LOAD'.format(description))
                    continue

        elif exchange == 'WIKIP_over2000':
            syms = pd.read_csv("C:/Users/awright/Desktop/January 2018/Other/trading/Asset listings/StockSymbols_over2000.csv"
                              , encoding = "ISO-8859-1")

            for i in range(len(syms)):  
                sym = syms['Symbol'][i]
                description = syms['Description'][i]
                try:
                    code = 'WIKI/' + sym
                    assetHistory = quandl.get(code, start_date=self.dateStart, end_date=self.dateEnd,collapse = period).reset_index()
                    assetHistory = assetHistory[['Date','Adj. Open','Adj. High','Adj. Low','Adj. Close','Adj. Volume']]
                    assetHistory['Symbol'] = sym
                    assetHistory['Description'] = description
                    assetHistory.columns = ['Date','Open','High','Low','Close', 'Volume','Symbol','Description']

                    self.History = pd.concat([self.History,assetHistory], ignore_index=True)
                    print('{} data acquired for dates from {} to {}'.format(description,self.dateStart,self.dateEnd))

                except (KeyError, NameError):
                    print('{} data FAILED TO LOAD'.format(description))
                    continue


        elif exchange == 'BNC3':
            for row in self.Dir[self.Dir['Exchange'] == exchange].index.tolist():
                try:
                    code = self.Dir.loc[row,'Listing']
                    assetHistory = quandl.get(code, start_date=self.dateStart, end_date=self.dateEnd,collapse = period).reset_index()
                    assetHistory = assetHistory[['Date','Open','High','Low','Close', 'Volume']]
                    assetHistory['Symbol'] = self.Dir.loc[row,'Symbol']
                    assetHistory['Description'] = self.Dir.loc[row,'Description']

                    self.History = pd.concat([self.History,assetHistory], ignore_index=True)
                    print('{} data acquired for dates from {} to {}'.format(self.Dir.loc[row,'Description'],self.dateStart,self.dateEnd))

                except (KeyError, NameError):
                    print('{} data FAILED TO LOAD'.format(self.Dir.loc[row,'Description']))
                    continue

        elif exchange == 'LME':
            for row in self.Dir[self.Dir['Exchange'] == exchange].index.tolist():
                code = self.Dir.loc[row,'Listing']
                try:
                    self.History = quandl.get(code, start_date=self.dateStart, end_date=self.dateEnd,collapse = period).reset_index()
                    print('London Metal Exchange Data Loaded for dates from {} to {} '.format(code,self.dateEnd))

                except (KeyError, NameError):
                    print('{} data FAILED TO LOAD'.format(code))
                    continue

        elif exchange == 'CME':
            listings = []
            for l in self.Dir[self.Dir['Exchange'] == 'CME'].loc[:,'Listing'].values.tolist():
                if l[-4:] in ['2017','2018','2019','2020','2021']:
                    listings.append(l)
                    if len(listings) > 2000:
                        listings = listings[:2000]

            for code in listings:
                description = self.Dir[self.Dir['Listing'] == code]['Description']
                try:
                    assetHistory = quandl.get(code, start_date=self.dateStart, end_date=self.dateEnd,collapse = period).reset_index()
                    assetHistory = assetHistory[['Date','Settle']]
                    assetHistory['Symbol'] = code[4:]
                    assetHisory['Description'] = description
                    assetHistory.columns = ['Date','Close','Symbol','Description']
                    self.History = pd.concat([self.History,assetHistory], ignore_index=True)
                    print('{} data acquired for dates from {} to {}'.format(description,self.dateStart,self.dateEnd))

                except (KeyError, NameError):
                    print('{} data FAILED TO LOAD'.format(description))
                    continue           

        elif exchange == 'ICE':
            listings = []
            for l in self.Dir[self.Dir['Exchange'] == 'ICE'].loc[:,'Listing'].values.tolist():
                if l[-4:] in ['2017','2018','2019','2020','2021']:
                    listings.append(l)
                    if len(listings) > 2000:
                        listings = listings[:2000]

            for code in listings:
                description = self.Dir[self.Dir['Listing'] == code]['Description']
                try:
                    assetHistory = quandl.get(code, start_date=self.dateStart, end_date=self.dateEnd,collapse = period).reset_index()
                    assetHistory = assetHistory[['Date','Settle']]
                    assetHistory['Symbol'] = code[4:]
                    assetHistory['Description'] = description
                    assetHistory.columns = ['Date','Close','Symbol', 'Description']
                    self.History = pd.concat([self.History,assetHistory], ignore_index=True)
                    print('{} data acquired for dates from {} to {}'.format(description,self.dateStart,self.dateEnd))

                except (KeyError, NameError):
                    print('{} data FAILED TO LOAD'.format(description))
                    continue

        elif exchange == 'LSE':
            syms = pd.read_csv("C:/Users/awright/Desktop/January 2018/Other/trading/Asset listings/LSE_Subset.csv")
            for i in rang(len(syms)):
                sym = syms['Symbol'][i]
                description = syms['Description'][i]
                try:
                    code = 'LSE/' + sym
                    assetHistory = quandl.get(code, start_date=self.dateStart, end_date=self.dateEnd,collapse = period).reset_index()
                    assetHistory = assetHistory[['Date','High','Low','Last Close', 'Volume']]
                    assetHistory['Symbol'] = sym
                    assetHistory['Description'] = description
                    assetHistory.columns = ['Date','High','Low','Close', 'Volume','Symbol','Description']

                    self.History = pd.concat([self.History,assetHistory], ignore_index=True)
                    print('{} data acquired for dates from {} to {}'.format(description,self.dateStart,self.dateEnd))

                except (KeyError, NameError):
                    print('{} data FAILED TO LOAD'.format(description))
                    continue

        elif exchange == 'COM':

            for row in self.Dir[self.Dir['Exchange'] == exchange].index.tolist():
                code = self.Dir.loc[row,'Listing']
                description = self.Dir.loc[row,'Description']
                try:
                    assetHistory = quandl.get(code, start_date=self.dateStart, end_date=self.dateEnd,collapse = period).reset_index()
                    assetHistory = assetHistory[['Date','Value']]
                    assetHistory['Symbol'] = self.Dir.loc[row,'Symbol']
                    assetHistory['Description'] = description
                    assetHistory.columns = ['Date','Close','Symbol','Description']
                    self.History = pd.concat([self.History,assetHistory], ignore_index=True)
                    print('{} data acquired for dates from {} to {}'.format(description,self.dateStart,self.dateEnd))

                except (KeyError, NameError):
                    print('{} data FAILED TO LOAD'.format(description))
                    continue

        elif exchange == 'HKEX':
            Symbols = []
            Codes = []
            for i in range(1,2001):
                digits = len(str(i))
                Symbol = '00000'
                Symbol = Symbol[:5-digits] + str(i)
                Code = 'HKEX/' + Symbol
                Symbols.append(Symbol)
                Codes.append(Code)

            for i in range(len(Codes)):
                description = self.Dir[self.Dir['Listing'] == Codes[i]]['Description']
                try:
                    assetHistory = quandl.get(Codes[i], start_date=self.dateStart, end_date=self.dateEnd,collapse = period).reset_index()
                    assetHistory = assetHistory[['Date','Nominal Price','High','Low','Share Volume (000)']]
                    assetHistory['Symbol'] = Symbols[i]
                    assetHistory['Description'] = description
                    assetHistory.columns = ['Date','Close','High','Low','Volume','Symbol','Description']
                    self.History = pd.concat([self.History,assetHistory], ignore_index=True)
                    print('{} data acquired for dates from {} to {}'.format(Description,self.dateStart,self.dateEnd))

                except (QuandlError, KeyError, NameError):
                    print('{} data FAILED TO LOAD'.format(description))
                    continue

        elif exchange == 'WSE':
            listings = []
            for l in self.Dir[self.Dir['Exchange'] == 'WSE'].loc[:,'Listing'].values.tolist():
                    listings.append(l)

            for code in listings:
                description = self.Dir[self.Dir['Listing'] == code]['Description']
                try:
                    assetHistory = quandl.get(code, start_date=self.dateStart, end_date=self.dateEnd,collapse = period).reset_index()
                    assetHistory = assetHistory[['Date','Open','High','Low','Close','Volume']]
                    assetHistory['Symbol'] = code[4:]
                    assetHistory['Description'] = description
                    assetHistory.columns = ['Date','Open','High','Low','Close','Volume','Symbol','Description']
                    self.History = pd.concat([self.History,assetHistory], ignore_index=True)
                    print('{} data acquired for dates from {} to {}'.format(description,self.dateStart,self.dateEnd))

                except (KeyError, NameError):
                    print('{} data FAILED TO LOAD'.format(description))
                    continue

        elif exchange == 'GPPG':
            listings = []
            for l in self.Dir[self.Dir['Exchange'] == 'GPP'].loc[:,'Listing'].values.tolist():
                    listings.append(l)

            for code in listings:
                description = self.Dir[self.Dir['Listing'] == code]['Description']
                try:
                    assetHistory = quandl.get(code, start_date=self.dateStart, end_date=self.dateEnd,collapse = period).reset_index()
                    assetHistory = assetHistory[['Date','Gasoline Price']]
                    assetHistory['Symbol'] = code[4:]
                    assetHistory['Description'] = description
                    assetHistory.columns = ['Date','Close','Symbol','Description']
                    self.History = pd.concat([self.History,assetHistory], ignore_index=True)
                    print('{} data acquired for dates from {} to {}'.format(description,self.dateStart,self.dateEnd))

                except (KeyError, NameError):
                    print('{} data FAILED TO LOAD'.format(description))
                    continue

        elif exchange == 'GPPD':
            listings = []
            for l in self.Dir[self.Dir['Exchange'] == 'GPP'].loc[:,'Listing'].values.tolist():
                    listings.append(l)
            for code in listings:
                description = self.Dir[self.Dir['Listing'] == code]['Description']
                try:
                    assetHistory = quandl.get(code, start_date=self.dateStart, end_date=self.dateEnd,collapse = period).reset_index()
                    assetHistory = assetHistory[['Date','Diesel Price']]
                    assetHistory['Symbol'] = code[4:]
                    assetHistory['Description'] = description
                    assetHistory.columns = ['Date','Close','Symbol','Description']
                    self.History = pd.concat([self.History,assetHistory], ignore_index=True)
                    print('{} data acquired for dates from {} to {}'.format(description,self.dateStart,self.dateEnd))

                except (KeyError, NameError):
                    print('{} data FAILED TO LOAD'.format(description))
                    continue

        elif exchange == 'LPPM':
            listings = []
            for l in self.Dir[self.Dir['Exchange'] == 'LPPM'].loc[:,'Listing'].values.tolist():
                    listings.append(l)

            for code in listings:
                description = self.Dir[self.Dir['Listing'] == code]['Description']
                try:
                    assetHistory = quandl.get(code, start_date=self.dateStart, end_date=self.dateEnd,collapse = period).reset_index()
                    assetHistory = assetHistory[['Date','USD PM']]
                    assetHistory['Symbol'] = code[5:]
                    assetHistry['Description'] = description
                    assetHistory.columns = ['Date','Close','Symbol','Description']
                    self.History = pd.concat([self.History,assetHistory], ignore_index=True)
                    print('{} data acquired for dates from {} to {}'.format(description,self.dateStart,self.dateEnd))

                except (KeyError, NameError):
                    print('{} data FAILED TO LOAD'.format(description))
                    continue

        elif exchange == 'LIFFE':
            listings = []
            for l in self.Dir[self.Dir['Exchange'] == 'LIFFE'].loc[:,'Listing'].values.tolist():
                if l[-4:] in ['2017','2018','2019','2020','2021','2022','2023']:
                    listings.append(l)

            for code in listings:
                description = self.Dir[self.Dir['Listing'] == code]['Description']
                try:
                    assetHistory = quandl.get(code, start_date=self.dateStart, end_date=self.dateEnd,collapse = period).reset_index()
                    assetHistory = assetHistory[['Date','Open','High','Low','Settle','Volume']]
                    assetHistory['Symbol'] = code[6:]
                    assetHistory['Description'] = description
                    assetHistory.columns = ['Date','Open','High','Low','Close','Volume','Symbol','Description']
                    self.History = pd.concat([self.History,assetHistory], ignore_index=True)
                    print('{} data acquired for dates from {} to {}'.format(description,self.dateStart,self.dateEnd))

                except (KeyError, NameError):
                    print('{} data FAILED TO LOAD'.format(description))
                    continue

        elif exchange == 'EUREX':
            listings = []
            for l in self.Dir[self.Dir['Exchange'] == 'EUREX'].loc[:,'Listing'].values.tolist():
                if l[-4:] in ['2017','2018','2019','2020','2021','2022','2023']:
                    listings.append(l)

            for code in listings:
                description = self.Dir[self.Dir['Listing'] == code]['Description']
                try:
                    assetHistory = quandl.get(code, start_date=self.dateStart, end_date=self.dateEnd,collapse = period).reset_index()
                    assetHistory = assetHistory[['Date','Open','High','Low','Settle','Volume']]
                    assetHistory['Symbol'] = code[6:]
                    assetHistory['Description'] = description
                    assetHistory.columns = ['Date','Open','High','Low','Close','Volume','Symbol','Description']
                    self.History = pd.concat([self.History,assetHistory], ignore_index=True)
                    print('{} data acquired for dates from {} to {}'.format(description,self.dateStart,self.dateEnd))

                except (KeyError, NameError):
                    print('{} data FAILED TO LOAD'.format(description))
                    continue

        elif exchange == 'JOHNMATT':
            listings = []
            for l in self.Dir[self.Dir['Exchange'] == 'JOHNMATT'].loc[:,'Listing'].values.tolist():
                    listings.append(l)

            for code in listings:
                description = self.Dir[self.Dir['Listing'] == code]['Description']
                try:
                    assetHistory = quandl.get(code, start_date=self.dateStart, end_date=self.dateEnd,collapse = period).reset_index()
                    assetHistory = assetHistory[['Date','New York 9:30']]
                    assetHistory['Symbol'] = code[9:]
                    assetHistory['Description'] = description
                    assetHistory.columns = ['Date','Close','Symbol','Description']
                    self.History = pd.concat([self.History,assetHistory], ignore_index=True)
                    print('{} data acquired for dates from {} to {}'.format(description,self.dateStart,self.dateEnd))

                except (KeyError, NameError):
                    print('{} data FAILED TO LOAD'.format(description))
                    continue
                    
        self.History = self.History[['Symbol','Description','Date','Volume','Open','Low','High','Close']]
        self.History['DailyPctChg'] = (a.History['Close'] / (a.History.groupby('Symbol')['Close'].shift(1))) - 1
        
   




    def tsPlotTogether(self,
                 symbols,
                  metric = 'price',
                 datestart = (str((datetime.datetime.now().date() + datetime.timedelta(weeks  =-4)).year) 
                      + '-' + str((datetime.datetime.now().date() + datetime.timedelta(weeks  =-4)).month) + '-' 
                      + str((datetime.datetime.now().date() + datetime.timedelta(weeks  =-4)).day)),
                  dateend = (str((datetime.datetime.now().date().year))
                      + '-' + str((datetime.datetime.now().date().month))
                      + '-' + str((datetime.datetime.now().date().day)))):
        ''' available metrics: price, maxnorm, pctchg'''
        

        dates = self.History['Date'][(self.History['Symbol'] == symbols[0]) 
                             &(self.History['Date'] >= datestart)
                             & (self.History['Date'] <= dateend)]
        
        fig, ax = plt.subplots(figsize=(16,8))
                               
        if metric == 'price':

            for i in range(len(symbols)):

                sym = symbols[i]
                prices = self.History[(self.History['Symbol'] == sym)
                           & (self.History['Date'] >= datestart)
                           & (self.History['Date'] <= dateend)].Close.values
                
                ax.set_title('Time Series from {} to {} \n Metric = {}'.format(datestart,dateend,metric), fontsize = 16)
                ax.plot(dates,prices, label = sym)

            plt.legend()
            plt.show()
            
        if metric == 'maxnorm':

            for i in range(len(symbols)):

                sym = symbols[i]
                prices = (self.History[(self.History['Symbol'] == sym)
                           & (self.History['Date'] >= datestart)
                           & (self.History['Date'] <= dateend)].Close.values 
                          / max(self.History[(self.History['Symbol'] == sym)
                           & (self.History['Date'] >= datestart)
                           & (self.History['Date'] <= dateend)].Close))
                          
                ax.set_title('Time Series from {} to {} \n Metric = {}'.format(datestart,dateend,metric), fontsize = 16)
                ax.plot(dates,prices, label = sym)

            plt.legend()
            plt.show()
            
        if metric == 'pctchg':

            for i in range(len(symbols)):

                sym = symbols[i]
                pctChgs = self.History[(self.History['Symbol'] == sym)
                           & (self.History['Date'] >= datestart)
                           & (self.History['Date'] <= dateend)].DailyPctChg.values
                
                ax.set_title('Time Series from {} to {} \n Metric = {}'.format(datestart,dateend,metric), fontsize = 16)
                ax.plot(dates,pctChgs, label = sym)

            plt.legend()
            plt.show()
            
    def tsPlotAlone(self,
                 symbols,
                  metric = 'price',
                 datestart = (str((datetime.datetime.now().date() + datetime.timedelta(weeks  =-4)).year) 
                      + '-' + str((datetime.datetime.now().date() + datetime.timedelta(weeks  =-4)).month) + '-' 
                      + str((datetime.datetime.now().date() + datetime.timedelta(weeks  =-4)).day)),
                  dateend = (str((datetime.datetime.now().date().year))
                      + '-' + str((datetime.datetime.now().date().month))
                      + '-' + str((datetime.datetime.now().date().day)))):
        ''' available metrics: price, pctchg'''
        


        
        ncols = 1
        nrows = len(symbols)
        fig, ax = plt.subplots(nrows = nrows, ncols = ncols,figsize=(16,8*nrows))
        
        if metric == 'price':

            for i in range(nrows):
                    sym = symbols[i]

                    dates = self.History['Date'][(self.History['Symbol'] == sym) 
                     &(self.History['Date'] >= datestart)
                     & (self.History['Date'] <= dateend)]

                    prices = self.History[(self.History['Symbol'] == sym)
                               & (self.History['Date'] >= datestart)
                               & (self.History['Date'] <= dateend)].Close.values
                    
                    volumes = self.History[(self.History['Symbol'] == sym)
                               & (self.History['Date'] >= datestart)
                               & (self.History['Date'] <= dateend)].Volume.values
                    
                    scale =  4 * ( max(self.History[(self.History['Symbol'] == sym)
                           & (self.History['Date'] >= datestart)
                           & (self.History['Date'] <= dateend)].Volume)
                            / max(self.History[(self.History['Symbol'] == sym)
                           & (self.History['Date'] >= datestart)
                           & (self.History['Date'] <= dateend)].Close))
                    
                    scaled_volumes = volumes / scale


                    ax[i].plot(dates,prices)
                    ax[i].bar(dates.tolist(),scaled_volumes, alpha = 0.7)
                    #ax[i].grid(which='both')
                    ax[i].set_title('{} from {} to {}\n Metric = {}'.format(sym,datestart,dateend,metric), fontsize=16)
                    

            plt.show()
            
        if metric == 'pctchg':

            for i in range(nrows):
                    sym = symbols[i]

                    dates = self.History['Date'][(self.History['Symbol'] == sym) 
                     &(self.History['Date'] >= datestart)
                     & (self.History['Date'] <= dateend)]

                    pctChgs = self.History[(self.History['Symbol'] == sym)
                               & (self.History['Date'] >= datestart)
                               & (self.History['Date'] <= dateend)].DailyPctChg.values
                    
                    volumes = self.History[(self.History['Symbol'] == sym)
                               & (self.History['Date'] >= datestart)
                               & (self.History['Date'] <= dateend)].Volume.values
                    
                    scale =  4 * ( max(self.History[(self.History['Symbol'] == sym)
                           & (self.History['Date'] >= datestart)
                           & (self.History['Date'] <= dateend)].Volume)
                            / max(self.History[(self.History['Symbol'] == sym)
                           & (self.History['Date'] >= datestart)
                           & (self.History['Date'] <= dateend)].DailyPctChg))
                    
                    scaled_volumes = volumes / scale


                    ax[i].plot(dates,pctChgs)
                    ax[i].bar(dates.tolist(),scaled_volumes, alpha = 0.7)
                    #ax[i].grid(which='both')
                    ax[i].set_title('{} from {} to {}\n Metric = {}'.format(sym,datestart,dateend,metric), fontsize=16)
                    
            plt.show()
            
         
    def tsPlotMavg(self,
                     symbols = [],
                     windows = [1,5,30],
                      metric = 'price',
                        mavg_df = False,
                     datestart = (str((datetime.datetime.now().date() + datetime.timedelta(weeks  =-4)).year) 
                          + '-' + str((datetime.datetime.now().date() + datetime.timedelta(weeks  =-4)).month) + '-' 
                          + str((datetime.datetime.now().date() + datetime.timedelta(weeks  =-4)).day)),
                      dateend = (str((datetime.datetime.now().date().year))
                          + '-' + str((datetime.datetime.now().date().month))
                          + '-' + str((datetime.datetime.now().date().day)))):
        
        
        
        
        ncols = 1
        nrows = len(symbols)
        fig, ax = plt.subplots(nrows = nrows, ncols = ncols, figsize=(16,8*nrows))
        
        if mavg_df == True:
            
            
        
        if metric == 'price':
            '''
            if mavg_df = True:
                
                symbols = self.History.Symbol.unique()
                
                for window in windows:
                    
            '''           
            
            for i in range(nrows):
                sym = symbols[i]
                
                
                dates = self.History['Date'][(self.History['Symbol'] == sym) 
                     &(self.History['Date'] >= datestart)
                     & (self.History['Date'] <= dateend)]

                prices = self.History[(self.History['Symbol'] == sym)
                           & (self.History['Date'] >= datestart)
                           & (self.History['Date'] <= dateend)].Close
                
                
                for window in windows:
                    
                    label = str(window) + ' day mavg'
                    mavgs = prices.rolling(window=window).mean().fillna(0)
                    
                    ax[i].plot(dates,mavgs, label = label)
                    ax[i].set_title('{} Rolling Averages from {} to {}\n Metric = {}'.format(sym,datestart,dateend,metric))
                    ax[i].legend()
                    
                    
            plt.legend()
            plt.show()
                
        
                        
                        
        
    def autocorr(self, symbols = [], days = 30, calc_only = False):
        
        if calc_only == True:
            
            symbols = self.History.Symbol.unique()
            acfs= []
            lags = range(1,days)
            for sym in symbols:
                sym_pctchg = a.History[a.History['Symbol'] == sym].DailyPctChg
                sym_acf = [sym_pctchg.autocorr(lag = i) for i in lags]
                acfs.append(sym_acf)

            self.acf_df = pd.DataFrame(acfs).T
            self.acf_df.columns = symbols
            self.acf_df.index += 1
        
        else:
            acfs= []
            lags = range(1,days)
            for sym in symbols:
                sym_pctchg = a.History[a.History['Symbol'] == sym].DailyPctChg
                sym_acf = [sym_pctchg.autocorr(lag = i) for i in lags]
                acfs.append(sym_acf)

            self.acf_df = pd.DataFrame(acfs).T
            self.acf_df.columns = symbols
            self.acf_df.index += 1

            ax = self.acf_df.plot(kind='bar'
                        , figsize=(16,8)
                        , grid=True
                        , position = 0
                        , title = 'Autocorrelation Corelleogram\n{}'.format(symbols)
                        )

            ax.set_xlabel('Lag (days)')
            ax.set_ylabel('r')
            plt.show()
            
    
    def saveHistory(self, path):
        '''C:/Users/awright/Desktop/January 2018/Other/trading/Market Histories'''
        self.History.to_csv(path)
        print('History saved\nFile located at {}'.format(path))
            

In [ ]:
Symbols = WIKIP2000.History.Symbol.unique()
windows = [5,30]
mavg_df = pd.DataFrame(WIKIP2000.History[(WIKIP2000.History['Date'] >= '2017-1-1') & (WIKIP2000.History['Date'] <= '2018-2-11')].Date)

mavg_dict = {'Date':WIKIP2000.History[(WIKIP2000.History['Date'] >= '2017-1-1') & (WIKIP2000.History['Date'] <= '2018-2-11')].Date.unique()}
for symbol in Symbols:
    
    for window in windows:
        
        label = symbol + str(window) + 'mavg'
        
        mavg_dict[label] = WIKIP2000.History[WIKIP2000.History['Symbol'] == symbol].Close.rolling(window=window).mean().values
        

In [ ]:
for i in range(len(b5)):
    
    date = b5.iloc[i].Date
    date1 = date.to_pydatetime()
    date2 = date1 + datetime.timedelta(days=-1)
    print(date1,date2)